In [40]:
import torch
import torch.nn as nn
from torch.nn import functional  as F
import pickle

block_size = 32
batch_size = 128
learning_rate = 2e-5
max_iters = 200
eval_iters = 100
n_embd = 384
n_layers = 4
n_head = 4
dropout = 0.2

with open('wizard_of_oz.txt', 'r', encoding= 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))

vocab_size = len(chars)

In [41]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype= torch.long)

In [42]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [43]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k]  = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [44]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads= nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out
        
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)
            
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.aa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.aa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layers)]) #4 Decoder Layers
        
        self.ln_f = nn.LayerNorm(n_embd) #Final Layer Norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self.__init__weights)
    def __init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
        
    def forward(self, index, targets=None):
        B, T = index.shape
        logits = self.token_embedding_table(index)
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = tok_emb + pos_emb
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)

#context = torch.zeros((1,1), dtype=torch.long)
#generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [45]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

tensor([165288,  73149,   7567, 128398,  49022,  22330,  47425,  48173,  86479,
         96580,  47385,  61077,   9510, 105476,  13320, 108786, 124831,  16607,
         49419,    398, 137551,  87227,  40575, 141754,  14122, 121074, 115036,
         62535, 124858,    544,  20438,  62608, 129335, 136400, 127001,  86336,
          4488,  22441,  62766,  21945, 135101, 150382,  43621, 155568, 163752,
         61155,  76280,  84464, 158717,  91776,  64824, 146141,   1377,  73503,
         78040,  58757,  64689,  21681,  52848, 109133, 101871, 155070, 161598,
         63894,  64874,  56433,  37189, 158841,  45364,  39991,  16800,  84352,
         56299,  26425,   4245,   6780, 109936, 158492,   4950, 112443,  16676,
        131396,  18423, 133942, 106492, 132843,  61623,  58715,  67109, 158708,
        143571,  80793,   5162, 163962,  18854, 100527,  66229,  62466,  80513,
         27783,  63316, 152440, 104945, 121160,  54216, 162063,  63170,  16992,
          8854, 152985,  98331,  50938, 

In [57]:
prompt = 'What is your Name ?'
context = torch.tensor(encode(prompt), dtype=torch.long)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=11)[0].tolist())
print(generated_chars)

What is your Name ?KgfIW9JAk99
